# Import de la base de dades i de les llibreries a utilitzar
---

## Llibreries
---

In [ ]:
import pandas as pd  
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

## Base de dades
---

In [ ]:
df = pd.read_csv('smartphone_data.csv')

In [ ]:
df

# ANÀLISIS I PREPROCESSAT DE DADES
---

In [ ]:
df.info()

In [ ]:
df['fast_charging_available'] = df['fast_charging_available'].astype(bool)
df['extended_memory_available'] = df['extended_memory_available'].astype(bool)

Per molt que `fast_charging` sembli una numèrica, realment és una categòrica ja que expressa el tipus de càrrega ràpida. Passa també amb moltes altres variables, com podrien ser `battery_size`, `num_cores`, `ram_capacity` o `screen_size`. Així doncs, tot i que aquestes variables haurien de ser tractades com a categòriques, ja que estan representades com a numèriques, les tractarem com a numèriques per a que el model predictiu pugui ser més precís i obtenir millors resultats, ja que el model que utilitzarem treballa millor amb variables numèriques.

In [ ]:
df = df.drop('model', axis=1)

Eliminem la variable ´model´ ja que no aporta cap informació rellevant per a la predicció.

## Analisi univariant de les dades
---

### Analisis de variables categoricas y booleanas

In [ ]:

def analisis_estadistic_cat_bool(df):
    for feature in df.columns:
        if df[feature].dtype == 'object' or df[feature].dtype == 'bool':
            # Calculamos el conteo y el porcentaje para cada categoría
            df_count = df[feature].value_counts().reset_index()
            df_count.columns = [feature, 'count']
            df_count['percent'] = 100 * df_count['count'] / df_count['count'].sum()

            # Creamos la gráfica de barras con colores distintos para cada categoría
            fig = px.bar(df_count, x=feature, y='count', text='percent', color=feature)

            # Actualizamos el layout para añadir título y etiquetas
            fig.update_layout(
                title=f'Distribución de la variable {feature}',
                xaxis_title=feature,
                yaxis_title='Conteo'
            )

            # Añadir el porcentaje en las barras
            fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

            fig.show()

In [ ]:
analisis_estadistic_cat_bool(df)

https://chat.openai.com/share/06002044-fa85-4c48-927b-a35ceb5b3d0a

In [ ]:
# Ver datos únicos para identificar formatos incorrectos
df['fast_charging'].unique()

asumim que 6n7.0 es un error tipografic i que realment volien dir 67.0 

In [ ]:

df['fast_charging'] = df['fast_charging'].replace('6n7.0', '67.0')

df['fast_charging'].unique()

In [ ]:
df['fast_charging'] = df['fast_charging'].astype(float)

In [ ]:
df['processor_brand'].unique()

In [ ]:
#cambiamos los valores 'sc9863a' por 'spreadtrum' en la columna 'processor_brand'
df['processor_brand'] = df['processor_brand'].replace('sc9863a', 'spreadtrum')

In [ ]:
df.info()

### Analisis variables numericas

In [ ]:
# Seleccionamos solo las columnas numéricas
numerical_columns = df.select_dtypes(include=['number'])

# Generamos la tabla de estadísticas descriptivas
statistics_table = numerical_columns.describe()

# Imprimimos la tabla
statistics_table.describe().round(2).T


In [ ]:
def analisis_estadistic_num(df, numerical_columns):
    for feature in numerical_columns:
        # Histograma
        fig = px.histogram(df, x=feature, marginal="box",
                        title=f'Histograma de {feature}')
        fig.update_layout(xaxis_title=feature, yaxis_title='Conteo')
        fig.show()

In [ ]:
analisis_estadistic_num(df, numerical_columns)

https://chat.openai.com/share/06002044-fa85-4c48-927b-a35ceb5b3d0a

## Analisi multivariant
---

In [ ]:
print("holaa")

## Estudi de balanceig de classes (documento)
---

Degut a que el objectiu es fer una regresio per predir el preu no fa falta

## Missings
---

In [ ]:
df.info()

In [ ]:
for feature in numerical_columns:
    # Calculem el nombre de valors absents
    missing_count = df[feature].isnull().sum()
    # Calculem el percentatge de valors absents
    missing_percentage = (missing_count / len(df)) * 100

    if missing_count > 0:
        print(f"La variable {feature} té {missing_count} valors absents, el que representa un {missing_percentage:.2f}%.")
    else:
        print(f"La variable {feature} no té valors absents.")

Antes de imputar tendremos que convertir a categoricas las variables que son categoricas pero estan representadas como numericas para evitar que se imputen valores incorrectos en las variables. Luego las volveremos a convertir a numericas para que el modelo las trate como tal.

## Outliers
---

INFORME

## Feature creation 
---

https://chat.openai.com/share/06002044-fa85-4c48-927b-a35ceb5b3d0a

### RESOLUTION


Variables Numéricas Separadas para Ancho y Alto:
*Proceso: Puedes separar la resolución indicada como "1080x2400" en dos variables numéricas distintas: una para el ancho (1080) y otra para el alto (2400). Esto te permite conservar la información sobre la dimensión de la pantalla de manera más específica y útil para el modelo.
*Ventajas: Al mantener estas dimensiones como numéricas, tu modelo puede aprender cómo diferentes resoluciones, ya sea más anchas o más altas, afectan al precio. Además, al separarlas, permites que el modelo ajuste independientemente el impacto del ancho y del alto.


separar las dimensiones permite al modelo aprender diferencias específicas en cómo el ancho y el alto afectan al precio, lo cual puede ser más informativo dado que ciertas proporciones de pantalla son más deseables en ciertos mercados o tendencias de diseño.

In [ ]:
# Crear nuevas columnas 'width' y 'height'
df[['width', 'height']] = df['resolution'].str.split('x', expand=True).astype(int)

df.drop('resolution', axis=1, inplace=True)

### fast

### extend memory

## Partició de les dades
---

### Mesclar

In [ ]:
df_shuffled = shuffle(df, random_state=69)

### Partició

In [ ]:

train, test = train_test_split(df_shuffled, test_size=0.30, random_state=69)
test, val = train_test_split(test, test_size=0.50, random_state=69)

## Tractament de missings
---

### Convertim a categoriques les variables que haurien de ser-ho per a la imputació

### Autoimputem missings a la db per veure els millors models per imputar

In [ ]:
# Funció per crear missings de manera controlada
def autoimpute_missing_values(data, missing_rate=0.1):
    df_missing = data.copy()
    for col in df_missing.columns:
        missing_indices = np.random.choice(df_missing.index, int(
            len(df_missing) * missing_rate), replace=False)
        # Reemplazar 'nan' con np.nan
        df_missing.loc[missing_indices, col] = np.where(
            df_missing.loc[missing_indices, col] == 'nan', np.nan, df_missing.loc[missing_indices, col])
    return df_missing

# Autoimputem missings
df_missing = autoimpute_missing_values(df)

In [ ]:
#convertim tots els nan a np.nan
df_missing = df_missing.replace('nan', np.nan)

In [ ]:
# Seleccionem només les columnes numèriques
numeric_cols = df_missing.select_dtypes(include=[np.number]).columns
df_missing_numeric = df_missing[numeric_cols]

# Funció per imputar valors numèrics amb RandomForest o HistGradientBoosting
def impute_numeric(data, cols, model_type='random_forest'):
    imputed_data = data.copy()
    for col in cols:
        # Comprova si hi ha files per imputar
        if imputed_data[col].isnull().sum() > 0:
            # Preparació del conjunt de dades
            train = imputed_data[imputed_data[col].notnull()]
            test = imputed_data[imputed_data[col].isnull()]
            y_train = train[col]
            X_train = train.drop(col, axis=1)
            X_test = test.drop(col, axis=1)

            # Omplir valors NaNs en X_train i X_test
            imputer = SimpleImputer(strategy='mean')
            X_train_imputed = imputer.fit_transform(X_train)
            X_test_imputed = imputer.transform(X_test)

            # Imputació amb el model seleccionat
            if model_type == 'random_forest':
                model = RandomForestRegressor(n_estimators=100)
            else:  # HistGradientBoosting
                model = HistGradientBoostingRegressor()

            model.fit(X_train_imputed, y_train)
            imputed_values = model.predict(X_test_imputed)

            # Assignació dels valors imputats
            imputed_data.loc[imputed_data[col].isnull(), col] = imputed_values


    return imputed_data



# Imputació amb RandomForest o HistGradientBoosting
df_numeric_rf = impute_numeric(df_missing_numeric, numeric_cols, model_type='random_forest')
df_numeric_hgb = impute_numeric(df_missing_numeric, numeric_cols, model_type='hist_gradient_boosting')

# Imputació amb MICE
mice_imputer = IterativeImputer()
df_numeric_mice = df_missing_numeric.copy()
imputed_data = mice_imputer.fit_transform(df_missing_numeric)

df_numeric_mice_imputed = pd.DataFrame(imputed_data, columns=df_missing_numeric.columns, index=df_missing_numeric.index)

In [ ]:

# Separació de les variables categòriques
categorical_cols = df_missing.select_dtypes(include=['object']).columns
df_missing_categorical = df_missing[categorical_cols]
df_missing_categorical = df_missing_categorical.replace('NaNN', np.nan)

# Funció per imputar valors categòrics
def impute_categorical(data, cols, method='most_frequent'):
    imputed_data = data.copy()
    for col in cols:
        imputer = SimpleImputer(strategy=method)
        imputed_data[col] = imputer.fit_transform(
            data[[col]]).ravel()  # Convertir en array 1D
    return imputed_data

# Funció per a Hot-Deck Imputation
def hot_deck_imputation(data, cols):
    imputed_data = data.copy()
    for col in cols:
        missing = imputed_data[col].isna()
        if missing.any():
            complete_mask = ~missing
            missing_mask = missing
            imputed_data.loc[missing_mask, col] = imputed_data.loc[complete_mask, col].sample(
                n=missing.sum(), replace=True).values
    return imputed_data


# Imputació amb Moda
df_categorical_mode = impute_categorical(
    df_missing_categorical, categorical_cols, method='most_frequent')

# Imputació amb Hot-Deck
df_categorical_hot_deck = hot_deck_imputation(
    df_missing_categorical, categorical_cols)

In [ ]:

def calculate_mse(original_data, imputed_data, cols):
    mse_scores = {}
    for col in cols:
        # Comparar només les files on original_data no té NaNs
        mask = original_data[col].notna()
        mse_scores[col] = mean_squared_error(original_data.loc[mask, col], imputed_data.loc[mask, col])
    return mse_scores


def calculate_accuracy(original_data, imputed_data, cols):
    accuracy_scores = {}
    for col in cols:
        # Reindexa imputed_data per coincidir amb original_data
        imputed_col = imputed_data[col].reindex(original_data.index)
        # Només compara on original_data no té NaN
        mask = original_data[col].notna()
        correct = original_data.loc[mask, col] == imputed_col.loc[mask]
        # Utilitza mean() per calcular la precisió
        accuracy_scores[col] = correct.mean()
    return accuracy_scores

### Provem quins metodes per imputar son els millors

### Imputem els missings amb els millors metodes

In [ ]:
# Separa les dades numèriques i categòriques per X_train
numeric_cols_train = train.select_dtypes(include=[np.number]).columns
categorical_cols_train = train.select_dtypes(include=['object', 'category']).columns

# Separa les dades numèriques i categòriques per X_test
numeric_cols_test = train.select_dtypes(include=[np.number]).columns
categorical_cols_test = train.select_dtypes(include=['object', 'category']).columns

df_numeric_train = train[numeric_cols_train]
df_categorical_train = train[categorical_cols_train].replace('NaNN', np.nan)

df_numeric_test = train[numeric_cols_test]
df_categorical_test = train[categorical_cols_test].replace('NaNN', np.nan)

# Utilitza la funció impute_numeric que has definit abans per X_train
df_numeric_train_imputed = impute_numeric(df_numeric_train, numeric_cols_train, model_type='random_forest')

# Utilitza la mateixa funció impute_numeric per X_test
df_numeric_test_imputed = impute_numeric(df_numeric_test, numeric_cols_test, model_type='random_forest')

# Utilitza la funció hot_deck_imputation que has definit abans per X_train
df_categorical_train_imputed = hot_deck_imputation(df_categorical_train, categorical_cols_train)

# Utilitza la mateixa funció hot_deck_imputation per X_test
df_categorical_test_imputed = hot_deck_imputation(df_categorical_test, categorical_cols_test)

# Uneix les dades numèriques i categòriques imputades per X_train
X_train_imputed = pd.concat([df_numeric_train_imputed, df_categorical_train_imputed], axis=1)

# Uneix les dades numèriques i categòriques imputades per X_test
X_test_imputed = pd.concat([df_numeric_test_imputed, df_categorical_test_imputed], axis=1)

In [ ]:
# Calcula els missings per a cada columna abans del preprocessament
missings_before_train = train.isna().sum()
missings_before_test = train.isna().sum()

# Calcula els missings per a cada columna després del preprocessament
missings_after_train = X_train_imputed.isna().sum()
missings_after_test = X_test_imputed.isna().sum()

# Imprimeix els resultats
print("Missings abans del preprocessament en X_train:")
print(missings_before_train)
print("\nMissings després del preprocessament en X_train:")
print(missings_after_train)

print("\nMissings abans del preprocessament en X_test:")
print(missings_before_test)
print("\nMissings després del preprocessament en X_test:")
print(missings_after_test)

### Tornem a convertir a numeriques les variables que tractarem com a tal

# MODELITZACIÓ


## Model lineal base
---

### Entrenar i avaluar
Entrenar i avaluar un model de regressió lineal o regressió logística segons la natura del problema (regressió o classificació). Entrenarem un model de regressió lineal per predir el preu dels dispositius mòbils, per tant el problema és de regressió.

### Interpretar els resultats obtinguts

## Procés iteratiu (MLP)
---

## Model guanyador i conclusions
---